# Manipulation.py: Shelf

In [1]:
import numpy as np
import robotic as ry
import manipulation as manip

In [2]:
C = ry.Config()
C.addFile(ry.raiPath('../rai-robotModels/scenarios/pandaSingle.g'))

C.delFrame("panda_collCameraWrist")
C.getFrame("table").setShape(ry.ST.ssBox, size=[1., 1., .1, .02])

# Shelf
pos = np.array([-.7, 0., 1.])
small_opening_dims = [.2, .2, .2]
inner_wall_width = .005

openings_small = [4, 6]

for i in range(openings_small[0]):
	for j in range(openings_small[1]):
		opening_pos = np.array([0, i*small_opening_dims[0], j*small_opening_dims[1]]) + pos - np.array([0, openings_small[0]*small_opening_dims[0]*.5, openings_small[1]*small_opening_dims[1]*.5]) + np.array([0, small_opening_dims[0]*.5, small_opening_dims[1]*.5])
		C.addFrame(f"small_box_left_{i}_{j}") \
			.setPosition(opening_pos - np.array([0., small_opening_dims[0]*.5, 0.])) \
			.setShape(ry.ST.ssBox, size=[small_opening_dims[2], inner_wall_width, small_opening_dims[1], 0.005]) \
			.setColor([1., 1., 0.]) \
			.setContact(1)
		C.addFrame(f"small_box_right_{i}_{j}") \
			.setPosition(opening_pos - np.array([0., -small_opening_dims[0]*.5, 0.])) \
			.setShape(ry.ST.ssBox, size=[small_opening_dims[2], inner_wall_width, small_opening_dims[1], 0.005]) \
			.setColor([1., 1., 0.]) \
			.setContact(1)
		C.addFrame(f"small_box_top_{i}_{j}") \
			.setPosition(opening_pos - np.array([0., 0., -small_opening_dims[1]*.5])) \
			.setShape(ry.ST.ssBox, size=[small_opening_dims[2], small_opening_dims[0], inner_wall_width, 0.005]) \
			.setColor([1., 1., 0.]) \
			.setContact(1)
		C.addFrame(f"small_box_bottom_{i}_{j}") \
			.setPosition(opening_pos - np.array([0., 0., small_opening_dims[1]*.5])) \
			.setShape(ry.ST.ssBox, size=[small_opening_dims[2], small_opening_dims[0], inner_wall_width, 0.005]) \
			.setColor([1., 1., 0.]) \
			.setContact(1)
		C.addFrame(f"small_box_blocker_{i}_{j}") \
			.setPosition(opening_pos - np.array([-small_opening_dims[0]*.5, 0., small_opening_dims[1]*.5-.015])) \
			.setShape(ry.ST.ssBox, size=[inner_wall_width, small_opening_dims[0], .03, 0.005]) \
			.setColor([1., 1., 0.]) \
			.setContact(1)
		
		C.addFrame(f"small_box_inside_{i}_{j}") \
			.setPosition(opening_pos) \
			.setShape(ry.ST.ssBox, size=[small_opening_dims[2], small_opening_dims[0], small_opening_dims[1], 0.005]) \
			.setColor([0., 0., 1., .5])
		
C.addFrame(f"shelf_back_{i}_{j}") \
	.setPosition(pos - np.array([small_opening_dims[2]*.5, 0, 0])) \
	.setShape(ry.ST.ssBox, size=[inner_wall_width, openings_small[0]*small_opening_dims[0], openings_small[1]*small_opening_dims[1], 0.005]) \
	.setColor([1., 1., 0.]) \
	.setContact(1)

# Objects
C.addFrame(f"box") \
	.setPosition([0, 0, .7]) \
	.setShape(ry.ST.ssBox, size=[.04, .04, .12, 0.005]) \
	.setColor([1., 0., 0.]) \
	.setContact(1) \
	.setMass(.1)

# for convenience, a few definitions:
gripper = "l_gripper"
palm = "l_palm"
table = "table"
box = "box"

C.view()

0

In [3]:
from time import sleep
def pick_place(shelf_coor: list[float], vis: bool=False) -> bool:
	box = "box"
	M = manip.ManipulationModelling(C, "shelf_task", helpers=[gripper])
	M.setup_pick_and_place_waypoints(gripper, box, homing_scale=1e-1)
	M.grasp_box(1., gripper, box, "l_palm", "x")
	M.komo.addObjective([2.], ry.FS.insideBox, [box, f"small_box_inside_{shelf_coor[0]}_{shelf_coor[1]}"], ry.OT.ineq, [1e1])
	
	keypoints = M.solve()
	if not M.feasible:
		M.komo.report(plotOverTime=True)
		C.view(True, f"{M.feasible}")
		return False, []
	
	for i, k in enumerate(keypoints):
		C.setJointState(k)
		if i%2:
			C.attach(table, box)
		else:
			C.attach(gripper, box)
		C.view(False, f'step {i}')
		sleep(1.)

	return True, keypoints

	M1 = M.sub_motion(0)
	M1.keep_distance([.3,.7], "l_palm", obj, margin=.05)
	M1.retract([.0, .2], gripper)
	M1.approach([.8, 1.], gripper)
	path1 = M1.solve()
	if not M1.feasible:
		return False, []

	M2 = M.sub_motion(1)
	M2.keep_distance([], table, "panda_collCameraWrist")
	M2.keep_distance([.2, .8], table, obj, .04)
	M2.keep_distance([], "l_palm", obj)
	path2 = M2.solve()
	if not M2.feasible:
		return False, []

	if vis:
		M1.play(C, 1.)
		C.attach(gripper, obj)
		M2.play(C, 1.)
		C.attach(table, obj)

	path = np.append(path1, path2, 0)
	return True, path

In [ ]:
shelf_coor = [2, 2]
success, _ = pick_place(shelf_coor, vis=True)